# Dashboard Visualization Algorithm

## Introduction


### Importation of Libraries

In [ ]:
import operator as op
import numpy as np
import pandas as pd
import requests
import param
import panel as pn
pn.extension('tabulator')
import hvplot.pandas

import hvplot.streamz
import holoviews as hv
from holoviews.element.tiles import EsriImagery
from holoviews.selection import link_selections
from datashader.utils import lnglat_to_meters
from streamz.dataframe import PeriodicDataFrame

ModuleNotFoundError: No module named 'panel'

In [ ]:
cd Senior Engineering Capstone

### Essential Functions

In [ ]:
from datetime import timedelta, date

def daterange(start, end):
  return [start + timedelta(n) for n in range(int((end - start).days))]

## Serial Data Reading, Cleaning, and Preparation 

In [ ]:
import serial
ser = serial.Serial('/dev/ttyACM0')
ser.flushInput()

while True:
    try:
        ser_bytes = ser.readline()
        decoded_bytes = float(ser_bytes[0:len(ser_bytes)-2].decode("utf-8"))
        print(decoded_bytes)
    except:
        print("Keyboard Interrupt")
        break

## Data Wrangling

### DataFrames Creation

In [1]:
# DataFrame for Normally Open and Closed Sensors
contactsensor = ['sensor1','sensor2']
sensorregion = ['Region 1','Region 2', 'Region 3', 'Region 4', 'Region 5', 'Region 6','Region 7']
data = np.random.randint(0,150,size=(33600, 2))
ocsensors = pd.DataFrame(data, index = [['Normally Open Sensor']*16800 + ['Normally Closed Sensor']*16800, sensorregion*4800],columns = [['ContactAngle']*1 + ['Position']*1])
ocsensors.index.names = ['Sensor Type','Sensor Region']
ocsensors.columns.names = ['Sensor Data']
ocsensors['Time'] = list(np.repeat(times,7))*2

In [10]:
times = pd.date_range(start='2023-03-10 00:00:00', end='2023-03-10 00:09:49', periods=2400).strftime('%H:%M:%S.%f')
times = times.to_list()

# DataFrame for BendLabs Sensor 
data_bl = times
blsensor = pd.DataFrame(data_bl, columns = ['Time']) 
blsensor['Stress'] = np.random.randint(50,100,size=(2400, 1))
blsensor['Strain'] = np.random.randint(0,50,size=(2400, 1))
blsensor.columns.names = ['Sensor Data']

Sensor Data,Time,Stress,Strain
0,00:00:00.000000,97,43
1,00:00:00.245518,68,4
2,00:00:00.491037,59,10
3,00:00:00.736556,67,11
4,00:00:00.982075,68,11
...,...,...,...
2395,00:09:48.017924,94,2
2396,00:09:48.263443,89,19
2397,00:09:48.508962,85,46
2398,00:09:48.754481,63,11


In [308]:
ocsensors.reset_index(inplace=True)
ocsensors.columns.name = None
ocsensors.columns = ['Sensor Type', 'Sensor Region','Contact Angle', 'Position', 'Time']
ocsensors

,Sensor Type,Sensor Region,Contact Angle,Position,Time
0,Normally Open Sensor,Region 1,64,122,00:00:00.000000
1,Normally Open Sensor,Region 2,118,3,00:00:00.000000
2,Normally Open Sensor,Region 3,72,69,00:00:00.000000
3,Normally Open Sensor,Region 4,58,14,00:00:00.000000
4,Normally Open Sensor,Region 5,66,129,00:00:00.000000
...,...,...,...,...,...
33595,Normally Closed Sensor,Region 3,20,140,00:09:49.000000
33596,Normally Closed Sensor,Region 4,4,91,00:09:49.000000
33597,Normally Closed Sensor,Region 5,70,26,00:09:49.000000
33598,Normally Closed Sensor,Region 6,126,99,00:09:49.000000


### DataFrames' Partition

### Creating Interactive Dashboard

In [85]:
# Make dataframe pipeline interactive
idf = flights_sample.interactive()

In [86]:
# Define Panel widgets
doy_slider = pn.widgets.IntSlider(name='Day_of_Year slider', start=0, end=365, step=1, value=200)
doy_slider

IntSlider(end=365, name='Day_of_Year slider', value=200)

In [87]:
# Radio buttons for Departure Delay Times
yaxis_dd = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options=['departure_delay', 'scheduled_departure',],
    button_type='success'
)

In [88]:
idf[(idf.day_of_year <= doy_slider)]

In [89]:
#continents = ['World', 'Asia', 'Oceania', 'Europe', 'Africa', 'North America', 'South America', 'Antarctica']

dd_pipeline = (
    idf[
        (idf.day_of_year <= doy_slider)
    ]
    .groupby(['origin_airport', 'day_of_year'])[yaxis_dd].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='departure_delay', ascending = False)  
    .reset_index(drop=True)
)

In [90]:
dd_pipeline

In [75]:
dd_plot = dd_pipeline.hvplot(x = 'day_of_year', by = 'state', y = yaxis_dd,line_width = 2, title = "Departure Delay by Airport")
dd_plot

KeyError: 'departure_delay'

# Algorithm Constituents

In [ ]:
// Example by Tom Igoe

import processing.serial.*;

Serial myPort;  // The serial port

void setup() {
  // List all the available serial ports:
  printArray(Serial.list());
  // Open the port you are using at the rate you want:
  myPort = new Serial(this, Serial.list()[0], 9600);
}

void draw() {
  while (myPort.available() > 0) {
    String inBuffer = myPort.readString();   
    if (inBuffer != null) {
      println(inBuffer);
    }
  }
}


## https://makersportal.com/blog/2018/2/25/python-datalogger-reading-the-serial-output-from-arduino-to-analyze-data-using-pyserial

### SERIAL READ LOOP

In [ ]:
import serial
ser = serial.Serial('/dev/ttyACM0')
ser.flushInput()

while True:
    try:
        ser_bytes = ser.readline()
        decoded_bytes = float(ser_bytes[0:len(ser_bytes)-2].decode("utf-8"))
        print(decoded_bytes)
    except:
        print("Keyboard Interrupt")
        break

### Saving Serial Data to .CSV File

In [ ]:
import serial
import time
import csv

ser = serial.Serial('/dev/ttyACM0')
ser.flushInput()

while True:
    try:
        ser_bytes = ser.readline()
        decoded_bytes = float(ser_bytes[0:len(ser_bytes)-2].decode("utf-8"))
        print(decoded_bytes)
        with open("test_data.csv","a") as f:
            writer = csv.writer(f,delimiter=",")
            writer.writerow([time.time(),decoded_bytes])
    except:
        print("Keyboard Interrupt")
        break

### LIVE PLOTTING SERIAL DATA USING MATPLOTLIB


In [ ]:
import serial
import time
import csv
import matplotlib
matplotlib.use("tkAgg")
import matplotlib.pyplot as plt
import numpy as np

ser = serial.Serial('/dev/ttyACM0')
ser.flushInput()

plot_window = 20
y_var = np.array(np.zeros([plot_window]))

plt.ion()
fig, ax = plt.subplots()
line, = ax.plot(y_var)

while True:
    try:
        ser_bytes = ser.readline()
        try:
            decoded_bytes = float(ser_bytes[0:len(ser_bytes)-2].decode("utf-8"))
            print(decoded_bytes)
        except:
            continue
        with open("test_data.csv","a") as f:
            writer = csv.writer(f,delimiter=",")
            writer.writerow([time.time(),decoded_bytes])
        y_var = np.append(y_var,decoded_bytes)
        y_var = y_var[1:plot_window+1]
        line.set_ydata(y_var)
        ax.relim()
        ax.autoscale_view()
        fig.canvas.draw()
        fig.canvas.flush_events()
    except:
        print("Keyboard Interrupt")
        break